In [1]:
import pandas as pd
from darts import TimeSeries
from darts.models import NHiTSModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import rmse, mae

# Load datatset for NHiTS prediction
df = pd.read_excel(r"C:\Users\sshan\Desktop\DS\Project 4\Mandi price nhits.xlsx", sheet_name="nhits")

# Parse MM-YYYY column as datetime (first of month)
df["date"] = pd.to_datetime(df["MM-YYYY"], format="%m-%Y")

# Sort and check continuity
df = df.sort_values("date").reset_index(drop=True)
print("Date range:", df["date"].min(), "to", df["date"].max())
print("Number of rows:", len(df))

# Check for missing months
expected_months = pd.date_range(df["date"].min(), df["date"].max(), freq="MS")
missing_months = expected_months.difference(df["date"])
print("Missing months:", missing_months)

# Choose target variable
target_col = "Modal Price Avg(Rs./MT)"  # <-- adjust name if needed
series = TimeSeries.from_dataframe(df, time_col="date", value_cols=target_col, freq="MS") #

# Covariates (all other features except target/date)
covariate_cols = [c for c in df.columns if c not in ["date","Month", "Year",target_col, "MM-YYYY"]]
covariates = TimeSeries.from_dataframe(df, time_col="date", value_cols=covariate_cols, freq="MS")

# Scale target and covariables
scaler_target = Scaler()
scaler_cov = Scaler()
series_scaled = scaler_target.fit_transform(series) # default scaling in darts is MinMax (0 to 1)
covariates_scaled = scaler_cov.fit_transform(covariates)

# Train/test split
train, val = series_scaled[:-5], series_scaled[-5:]
cov_train, cov_val = covariates_scaled[:-5], covariates_scaled[-5:]

# Initialize N-HiTS
model = NHiTSModel(
    input_chunk_length=12,  # look back 12 months
    output_chunk_length=6,  # predict 6 months
    n_epochs=200,
    random_state=42
)

# Fit the model
model.fit(series=series_scaled, past_covariates=covariates_scaled, verbose=True)

# Forecast next 6 months
forecast = model.predict(n=6, series = series_scaled, past_covariates=covariates_scaled)

# Evaluate
print("RMSE:", rmse(val, forecast))
print("MAE:", mae(val, forecast))

# Inverse transform forecast for real values
forecast_real = scaler_target.inverse_transform(forecast)
val_real = scaler_target.inverse_transform(val)

print("Validation actual vs forecast:")
print(pd.DataFrame({"forecast": forecast_real.values().flatten()}))

GPU available: False, used: False


Date range: 2019-04-01 00:00:00 to 2025-08-01 00:00:00
Number of rows: 77
Missing months: DatetimeIndex([], dtype='datetime64[ns]', freq='MS')


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 1.6 M 
---------------------------------------------------
1.4 M     Trainable params
153 K     Non-trainable params
1.6 M     Total params
6.337     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: nan
MAE: nan
Validation actual vs forecast:
       forecast
0  28896.107921
1  31508.250948
2  34158.505011
3  34920.204025
4  32998.796113
5  34608.183261


C:\Users\sshan\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\sshan\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\sshan\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\sshan\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
